## WRM stream analysis

Ten notatnik ma na celu przybliżenie problematyki projektu przetwarza danych strumieniowych z systemu udostępniania rowerów WRM (Wrocławskiego Roweru Miejskiego), obecnie zarządzanym przez firmę nextbike.

Projekt składać będzie się z kilku elementów:
- Skryptu pobierającego dane, uniwersalnego dla obu typu danych
- Skryptu przetwarzającego dane WRM
- Skryptu przetwarzającego dane pogodowe
- Modelu przewidującego ilość rowerów w zależności od pory dnia oraz pogody

Planowane jest zbieranie danych co 5 minut

poniżej definicja adresów

In [ ]:
bike_data_api = "https://api.nextbike.net/maps/nextbike-live.json"
imgw_weather_api = "nothing_yet"  # Wrocław

### WRM Data collection description

Dane pobierane są z get https://api.nextbike.net/maps/nextbike-live.json?city=(city_id)
gdzie city_id dla Wrocławia to 148

API nextbike w odpowiedzi zwraca json z danymi miast, wszystkich stacji oraz rowerów dostępnych w systemie
dodanie parametru city zwraca powoduje zwrot danych tylko dla miasta o podanym id.

W mojej analizie interesują mnie poniższe parametry:
- places - lista obiektów stacji z rowerami
- places.uid - id stacji
- places.lat + places.lng - współrzędne geograficzne stacji
- places.bikes - ilość rowerów na stacji
- places.available_bikes - dostępne rowery 

Poniżej użycie data collectora dla wrm:

In [ ]:
from data_collector import collect_data

bike_data = collect_data(bike_data_api, {"city": 148})

No i zbadanie wyniku

In [ ]:
bike_data

{'countries': [{'lat': 51.1093,
   'lng': 17.0372,
   'zoom': 11,
   'name': 'WRM nextbike Poland',
   'hotline': '+48(71)7381111',
   'domain': 'pl',
   'language': 'pl',
   'email': 'ck@wroclawskirower.pl',
   'timezone': 'Europe/Warsaw',
   'currency': 'PLN',
   'country_calling_code': '+48',
   'system_operator_address': 'nextbike GmbH, Erich-Zeigner Allee 69-73, 04229 Leipzig, Germany',
   'country': 'PL',
   'country_name': 'Poland',
   'terms': 'https://www.wroclawskirower.pl/terms',
   'policy': 'https://www.wroclawskirower.pl/polityka-prywatnosci',
   'website': 'https://www.wroclawskirower.pl/',
   'show_bike_types': False,
   'show_bike_type_groups': False,
   'show_free_racks': False,
   'booked_bikes': 0,
   'set_point_bikes': 5481,
   'available_bikes': 2504,
   'capped_available_bikes': False,
   'no_registration': False,
   'pricing': 'https://www.wroclawskirower.pl/',
   'vat': '23',
   'faq_url': 'https://www.wroclawskirower.pl/',
   'store_uri_android': 'https://play

Następnie przeprowadzamy wydobywamy potrzebne dla nas dane i przekazujemy je do csv

In [ ]:
bike_data['countries'][0]['cities'][0]['places']

[{'uid': 12497516,
  'lat': 51.108004,
  'lng': 17.039528,
  'bike': False,
  'name': 'Plac Dominikański (Galeria Dominikańska)',
  'address': None,
  'spot': True,
  'number': 15001,
  'booked_bikes': 0,
  'bikes': 7,
  'bikes_available_to_rent': 7,
  'active_place': 1,
  'bike_racks': 16,
  'free_racks': 9,
  'special_racks': 0,
  'free_special_racks': 0,
  'maintenance': False,
  'terminal_type': '7inch',
  'bike_list': [{'number': '592760',
    'bike_type': 71,
    'lock_types': ['frame_lock'],
    'active': True,
    'state': 'ok',
    'electric_lock': True,
    'boardcomputer': 83300034062,
    'pedelec_battery': None,
    'battery_pack': None},
   {'number': '592017',
    'bike_type': 71,
    'lock_types': ['frame_lock'],
    'active': True,
    'state': 'ok',
    'electric_lock': True,
    'boardcomputer': 83300036721,
    'pedelec_battery': None,
    'battery_pack': None},
   {'number': '591691',
    'bike_type': 71,
    'lock_types': ['frame_lock'],
    'active': True,
    's

In [ ]:
from wrm_data_processing import process_wrm_data
processed_bike_data = process_wrm_data(bike_data)

In [ ]:
processed_bike_data

[{'id': 12497516,
  'name': 'Plac Dominikański (Galeria Dominikańska)',
  'latitude': 51.108004,
  'longitude': 17.039528,
  'bikes_total': 7,
  'bikes_available': 7},
 {'id': 12497641,
  'name': 'Dworzec Główny, południe',
  'latitude': 51.097108,
  'longitude': 17.03611,
  'bikes_total': 3,
  'bikes_available': 3},
 {'id': 12497699,
  'name': 'Rynek',
  'latitude': 51.109782,
  'longitude': 17.030175,
  'bikes_total': 16,
  'bikes_available': 16},
 {'id': 12497748,
  'name': 'Dworzec Główny ',
  'latitude': 51.09975,
  'longitude': 17.036228,
  'bikes_total': 0,
  'bikes_available': 0},
 {'id': 12497776,
  'name': 'Nowowiejska / Jedności Narodowej',
  'latitude': 51.124879,
  'longitude': 17.045844,
  'bikes_total': 1,
  'bikes_available': 1},
 {'id': 12497800,
  'name': 'Legnicka / Wejherowska',
  'latitude': 51.125276,
  'longitude': 16.984447,
  'bikes_total': 4,
  'bikes_available': 4},
 {'id': 12497838,
  'name': 'Rondo Reagana',
  'latitude': 51.112154,
  'longitude': 17.060451